In [1]:

import re
import os
import sys
import boto3 as bt3
import s3fs
import pandas as pd
# import qgrid


In [2]:
from pyspark import *
from pyspark.sql import *
from IPython.display import display


In [3]:

pd.set_option('display.max_columns', 500)
pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.max_colwidth', -1)



In [4]:

# ====================================
# Data Loading

s3 = bt3.resource('s3')
my_bucket = s3.Bucket('tc-jp-bucket1')
s3_loc = 's3://tc-jp-bucket1'
data_loc_list = ['scratch/',
  ]



In [5]:
# Visual list of files
ls_loc = []
ls_file = []
for data_loc in data_loc_list:
  for obj in my_bucket.objects.filter(Prefix=data_loc):
    print(obj.key)
    file = s3_loc + obj.key
    ls_file.append(file)
  print(data_loc + '\n')
print("number of files:" + str(len(ls_file)))



scratch/
scratch/new_purchases.csv
scratch/new_purchases.parquet
scratch/

number of files:3


In [15]:
spark = SparkSession.builder \
.appName('TC-APP1') \
.master('local[*]') \
.config('spark.sql.hive.convertMetastoreParquet', 'false') \
.config('spark.sql.hive.caseSensitiveInferenceMode', 'NEVER_INFER') \
.enableHiveSupport() \
.getOrCreate()


spark.sparkContext
     .hadoopConfiguration.set("fs.s3a.access.key", "awsaccesskey value")
service)
 // Replace Key with your AWS secret key (You can find this on IAM 
spark.sparkContext
     .hadoopConfiguration.set("fs.s3a.secret.key", "aws secretkey value")
spark.sparkContext
      .hadoopConfiguration.set("fs.s3a.endpoint", "s3.amazonaws.com")


# SparkConf().getAll()



[('spark.sql.hive.convertMetastoreParquet', 'false'),
 ('spark.sql.hive.caseSensitiveInferenceMode', 'NEVER_INFER'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'TC-APP1')]

In [23]:
# Read Jupyter File data

df1 = spark.read.format("com.databricks.spark.csv") \
.load("data/customers.csv")

tables = spark.catalog.listTables()
display(tables)


[]

AnalysisException: 'Table or view not found: TimeStamp; line 1 pos 14'

In [11]:
# spark.sql("CREATE DATABASE IF NOT EXISTS dbnorth COMMENT 'This is DB for northwind data' ").show()
# spark.sql("DESCRIBE DATABASE EXTENDED dbnorthwind ").show()



In [14]:
# Pandas load

my_bucket 
my_bucket = ["s3://tc-jp-bucket1/"]
my_file = ["northwind-db/customers.csv"]
data_location = 's3://{}/{}'.format(my_bucket,my_file)
data=pd.read_csv(data_location)
data.head(2)

# df_target1 = spark.read.format("csv").load(v_source_file_loc_path)




ValueError: Invalid IPv6 URL

In [13]:
# v_source_file_loc_path = ["s3://tc-jp-bucket1/northwind-db/customers.csv"]
# df_target1 = spark.read.format("csv").load(v_source_file_loc_path)

df_target1 = spark.read.csv("s3://tc-jp-bucket1/northwind-db/customers.csv")
df_target1.printSchema()


Py4JJavaError: An error occurred while calling o252.csv.
: java.io.IOException: No FileSystem for scheme: s3
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.datasources.DataSource$.org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary(DataSource.scala:709)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$15.apply(DataSource.scala:390)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$15.apply(DataSource.scala:390)
	at scala.collection.TraversableLike$$anonfun$fl

In [ ]:

# Temp tables are more resistant to a shared hive tables - avoids tampering
df_target1.registerTempTable("t_check_attrib")

df_target1.columns
display(df_target1.columns)